In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import re
from bs4 import BeautifulSoup

1 - Acquisition

    DataBase Conection

In [2]:
def db_connection(db_path):
    print(f'Connecting to data base {db_path}')
    print("...")
    print("...")
    connection = create_engine(f'sqlite:////{db_path}')
    return connection

In [3]:
def sql_query_to_df(table, data_base):
    
    select_all_query = pd.read_sql_query(f'SELECT * FROM {table}', data_base)
    return select_all_query

    DataBase tables to DataFrames

In [4]:
def tables_to_df(arguments):  # Exec number 1
    db_path = arguments
    data_base = db_connection(db_path)
    table_names = data_base.table_names()
    print(f"Obtaining tables from data base provided")
    print("...")
    print("...")

    df_list = []

    for table in table_names:
        sql_query = sql_query_to_df(table, data_base)
        print(f"Converting '{table}' table into data frame")
        df_list.append(sql_query)

    db_df = pd.DataFrame(df_list[0])
    for df in df_list[1:]:
        db_df = db_df.merge(df, left_on='uuid', right_on='uuid')
    print("...")
    print("...")
    print("Merging all tables...")
    print("...")
    print("...")
    return db_df

In [5]:
db_path = 'home/david/Documents/ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db'
a = tables_to_df(db_path)

Connecting to data base home/david/Documents/ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db
...
...
Obtaining tables from data base provided
...
...
Converting 'career_info' table into data frame
Converting 'country_info' table into data frame
Converting 'personal_info' table into data frame
Converting 'poll_info' table into data frame
...
...
Merging all tables...
...
...


In [6]:
a.columns

Index(['uuid', 'dem_education_level', 'dem_full_time_job',
       'normalized_job_code', 'country_code', 'rural', 'age', 'gender',
       'dem_has_children', 'age_group',
       'question_bbi_2016wave4_basicincome_awareness',
       'question_bbi_2016wave4_basicincome_vote',
       'question_bbi_2016wave4_basicincome_effect',
       'question_bbi_2016wave4_basicincome_argumentsfor',
       'question_bbi_2016wave4_basicincome_argumentsagainst'],
      dtype='object')

    Get job titles from API

In [7]:
def get_job_titles(url, json_acum=[]):
    
    print(f'Getting job titles from API {url}')
    response = requests.get(url)
    json = response.json()
    json_acum.append(json[:-1])

    root = 'http://api.dataatwork.org/v1'
   
    for elem in json[-1]['links']:
        if elem['rel'] == 'next':
            link = elem['href']
            next_link = f'{root}{link}'
            get_job_titles(next_link, json_acum)
        
    return json_acum

In [8]:
def job_titles_to_DataFrame():
    url = 'http://api.dataatwork.org/v1/jobs?limit=500'
    json = get_job_titles(url)
    
    jobs_df = []
    for result in json:
        jobs_df.extend(result)
        
    jobs_df = pd.DataFrame(jobs_df)
    jobs_df.rename(columns={'uuid': 'normalized_job_code'}, inplace=True)
    
    return jobs_df

    Country names from Web Scrapping

In [9]:
def get_country_names():
    url = 'https://www.iban.com/country-codes'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    items = soup.find_all('tr')

    a = str(items).split('<td>')

    paises = []
    for elem in a[1:]:
        match = re.search('(\w+\s\w+)', elem)
        if match:
            paises.append(re.findall('(\w+\s\w+)', elem))
        else:
            paises.append(re.findall('\w+', elem))

    countries = paises[0::4]
    codes = paises[1::4]
    
    country_codes = []
    for index, elem in enumerate(countries):
        country_codes.append([elem[0], codes[index][0]])
    country_codes = pd.DataFrame(country_codes, columns=['Country', 'country_code'])
    country = country_codes[['Country']].drop_duplicates()
    code = country_codes[['country_code']].drop_duplicates()
    country_codes = pd.merge(country, code, left_index=True, right_index=True)
    
    return country_codes

In [10]:
arguments = 'home/david/Documents/ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db'


In [11]:
def merge_data(arguments):
    db_tables = tables_to_df(arguments)
    job_titles = job_titles_to_DataFrame()
    country_names = get_country_names()
    
    print("Creating final DataFrame")
    main_df = pd.merge(db_tables, job_titles, on='normalized_job_code', how='left')
    print("...")
    print("...")
    print("Adding Job titles to final DataFrame")
    print("...")
    print("...")
    main_df2 = pd.merge(main_df, country_names, on='country_code', how='left')
    print("...")
    print("...")
    print("Adding country names to final DataFrame")
    print("...")
    print("...")
    #db_df.merge()
    
    return main_df2

In [12]:
main_df = merge_data(arguments)


Connecting to data base home/david/Documents/ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db
...
...
Obtaining tables from data base provided
...
...
Converting 'career_info' table into data frame
Converting 'country_info' table into data frame
Converting 'personal_info' table into data frame
Converting 'poll_info' table into data frame
...
...
Merging all tables...
...
...
Getting job titles from API http://api.dataatwork.org/v1/jobs?limit=500
Getting job titles from API http://api.dataatwork.org/v1/jobs?offset=500&limit=500
Getting job titles from API http://api.dataatwork.org/v1/jobs?offset=1000&limit=500
Getting job titles from API http://api.dataatwork.org/v1/jobs?offset=1500&limit=500
Getting job titles from API http://api.dataatwork.org/v1/jobs?offset=2000&limit=500
Getting job titles from API http://api.dataatwork.org/v1/jobs?offset=2500&limit=500
Getting job titles from API http://api.dataatwork.org/v1/jobs?offset=3000&limit=500
Getting job titles from API http://a

In [13]:
main_df

,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,title,normalized_job_title,parent_uuid,Country
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None,AT,countryside,61 years old,male,NO,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above,NaN,NaN,NaN,Austria
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,urban,57 years old,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,Austria
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None,AT,city,32 years old,male,nO,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,NaN,NaN,NaN,Austria
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,Country,45 years old,Male,YES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Data Coordinator,data coordinator,0b9dd32a367f4562ec77b993053d1910,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,city,41 years old,Fem,yES,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Database Developer,database developer,b90ca4df5690002377a7b0f1f3d40781,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,urban,37 years old,FeMale,nO,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,Slovakia
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,urban,53 years old,Male,yES,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,Database Manager,database manager,b90ca4df5690002377a7b0f1f3d40781,Slovakia
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None,SK,Non-Rural,1992,male,NO,juvenile,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,NaN,NaN,NaN,Slovakia
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,city,47 years old,male,yES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,Data Officer,data officer,596226d5f9ef63de41a852826d483d19,Slovakia


In [14]:
main_df = main_df[[
    'dem_education_level', 'dem_full_time_job', 'rural', 'age', 'gender', 'dem_has_children', 'age_group', 
    'question_bbi_2016wave4_basicincome_awareness', 'question_bbi_2016wave4_basicincome_vote', 
    'question_bbi_2016wave4_basicincome_effect', 'question_bbi_2016wave4_basicincome_argumentsfor', 
    'question_bbi_2016wave4_basicincome_argumentsagainst', 'normalized_job_title', 'Country']]

In [15]:
main_df.columns = [
    'Education_level', 'Full_time_job', 'Living area', 'Age', 'Gender', 'Children', 'Age_group', 'Question_basicincome_awareness', 
    'Question_basicincome_vote', 'Question_basicincome_effect', 'Question_basicincome_argumentsfor', 'Question_basicincome_argumentsagainst', 
    'Job_title', 'Country']

In [16]:
main_df

,Education_level,Full_time_job,Living area,Age,Gender,Children,Age_group,Question_basicincome_awareness,Question_basicincome_vote,Question_basicincome_effect,Question_basicincome_argumentsfor,Question_basicincome_argumentsagainst,Job_title,Country
0,no,no,countryside,61 years old,male,NO,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above,NaN,Austria
1,high,yes,urban,57 years old,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,Austria
2,None,no,city,32 years old,male,nO,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,NaN,Austria
3,high,yes,Country,45 years old,Male,YES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,data coordinator,Austria
4,high,yes,city,41 years old,Fem,yES,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,database developer,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,high,yes,urban,37 years old,FeMale,nO,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,data warehouse developer,Slovakia
9645,high,yes,urban,53 years old,Male,yES,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,database manager,Slovakia
9646,low,no,Non-Rural,1992,male,NO,juvenile,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,NaN,Slovakia
9647,low,yes,city,47 years old,male,yES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,data officer,Slovakia


In [17]:
main_df.columns

Index(['Education_level', 'Full_time_job', 'Living area', 'Age', 'Gender',
       'Children', 'Age_group', 'Question_basicincome_awareness',
       'Question_basicincome_vote', 'Question_basicincome_effect',
       'Question_basicincome_argumentsfor',
       'Question_basicincome_argumentsagainst', 'Job_title', 'Country'],
      dtype='object')

In [18]:
len(main_df)

9649

In [19]:
sub_df = main_df[['Age_group','Job_title', 'Country']]
sub_df


,Age_group,Job_title,Country
0,40_65,NaN,Austria
1,40_65,automatic data processing planner,Austria
2,26_39,NaN,Austria
3,40_65,data coordinator,Austria
4,40_65,database developer,Austria
...,...,...,...
9644,26_39,data warehouse developer,Slovakia
9645,40_65,database manager,Slovakia
9646,juvenile,NaN,Slovakia
9647,40_65,data officer,Slovakia


In [20]:
percentage = [100/len(sub_df) for e in range(len(sub_df))]
quantity = [1 for e in range(len(sub_df))]

In [21]:
sub_df['Quantity'] = quantity
sub_df['Percentage'] = percentage

/home/david/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/david/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
sub_df

,Age_group,Job_title,Country,Quantity,Percentage
0,40_65,NaN,Austria,1,0.010364
1,40_65,automatic data processing planner,Austria,1,0.010364
2,26_39,NaN,Austria,1,0.010364
3,40_65,data coordinator,Austria,1,0.010364
4,40_65,database developer,Austria,1,0.010364
...,...,...,...,...,...
9644,26_39,data warehouse developer,Slovakia,1,0.010364
9645,40_65,database manager,Slovakia,1,0.010364
9646,juvenile,NaN,Slovakia,1,0.010364
9647,40_65,data officer,Slovakia,1,0.010364


In [23]:
sub_df = sub_df[['Country', 'Age_group', 'Job_title', 'Quantity', 'Percentage']]

In [60]:
prueba = sub_df.groupby(['Country', 'Age_group', 'Job_title']).sum().reset_index()
prueba

,Country,Age_group,Job_title,Quantity,Percentage
0,Austria,14_25,data communications software consultant,1,0.01
1,Austria,14_25,data officer,1,0.01
2,Austria,14_25,data processing auditor,3,0.03
3,Austria,14_25,data warehousing engineer,1,0.01
4,Austria,14_25,database development and administration projec...,1,0.01
...,...,...,...,...,...
2916,United Kingdom,juvenile,database engineer,1,0.01
2917,United Kingdom,juvenile,electronic data interchange system developer e...,1,0.01
2918,United Kingdom,juvenile,field traffic data technician,1,0.01
2919,United Kingdom,juvenile,geographic information systems database admini...,1,0.01


In [37]:
sub_df[['Percentage']] = sub_df[['Percentage']].applymap('{:,.2f}'.format)

ValueError: Unknown format code 'f' for object of type 'str'

In [41]:
sub_df.dtypes
sub_df['Percentage'] = sub_df['Percentage'].astype(float)

In [65]:
main_df.columns

Index(['Education_level', 'Full_time_job', 'Living area', 'Age', 'Gender',
       'Children', 'Age_group', 'Question_basicincome_awareness',
       'Question_basicincome_vote', 'Question_basicincome_effect',
       'Question_basicincome_argumentsfor',
       'Question_basicincome_argumentsagainst', 'Job_title', 'Country'],
      dtype='object')

In [68]:
main_df['Question_basicincome_argumentsfor'].unique()

array(['None of the above',
       'It increases appreciation for household work and volunteering | It encourages financial independence and self-responsibility | It reduces anxiety about financing basic needs',
       'It creates more equality of opportunity',
       'It reduces anxiety about financing basic needs',
       'It encourages financial independence and self-responsibility | It reduces anxiety about financing basic needs',
       'It increases appreciation for household work and volunteering | It reduces anxiety about financing basic needs',
       'It creates more equality of opportunity | It reduces anxiety about financing basic needs | It reduces bureaucracy and administrative expenses',
       'It increases solidarity, because it is funded by everyone',
       'It increases appreciation for household work and volunteering',
       'It reduces anxiety about financing basic needs | It creates more equality of opportunity | It encourages financial independence and self-res